## JTK_CYCLE

1. Make sure the JTK_CYCLEv3 script is in the working directory
2. Split the normalized counts into 2 worksheets, i.e. WT_only and KO_only
3. Arrange counts from each library (each column) by timepoints, also clustering replicates together
4. The first column should be gene identifier (gene name, ENSEMBL ID etc.)
5. Make an annotation file that contain the list of gene identifier (seems redundant to me but the rules are rules)
6. Change the `project` to a name you like. `annot` and `data` should be set to the annotation file and arranged normalized counts file
7. Set `jtkdist(x,y)`, where `x` is number of timepoints and `y` is number of replicates per timepoint
8. Set `periods <- p,q`, where `p` is (minimum period you want to measure)/(timepoint interval) and `q` is (maximum period you want to measure)/(timepoint interval)
9. Set the `z` in `jtk.init(periods, z)` to timepoint interval

In [ ]:
source("JTK_CYCLEv3.1.R")

project <- "JTK_wt"

options(stringsAsFactors=FALSE)
annot <- read.delim("annoJTK1.txt")
data <- read.delim("wtcountsJTK.txt")

rownames(data) <- data[,1]
data <- data[,-1]
jtkdist(4,5) 	# 4 total time points, 5 replicates per time point

periods <- 2:4
jtk.init(periods,6)

cat("JTK analysis started on",date(),"\n")
flush.console()

st <- system.time({
  res <- apply(data,1,function(z) {
    jtkx(z)
    c(JTK.ADJP,JTK.PERIOD,JTK.LAG,JTK.AMP)
  })
  res <- as.data.frame(t(res))
  bhq <- p.adjust(unlist(res[,1]),"BH")
  res <- cbind(bhq,res)
  colnames(res) <- c("BH.Q","ADJ.P","PER","LAG","AMP")
  results <- cbind(annot,res,data)
  results <- results[order(res$ADJ.P,-res$AMP),]
})
print(st)

save(results,file=paste("JTK",project,"rda",sep="."))
write.table(results,file=paste("JTK",project,"txt",sep="."),row.names=F,col.names=T,quote=F,sep="\t")

In [ ]:
source("JTK_CYCLEv3.1.R")

project <- "JTK_ko"

options(stringsAsFactors=FALSE)
annot <- read.delim("annoJTK1.txt")
data <- read.delim("kocountsJTK.txt")

rownames(data) <- data[,1]
data <- data[,-1]
jtkdist(4,5) 	# 4 total time points, 5 replicates per time point

periods <- 2:4
jtk.init(periods,6)

cat("JTK analysis started on",date(),"\n")
flush.console()

st <- system.time({
  res <- apply(data,1,function(z) {
    jtkx(z)
    c(JTK.ADJP,JTK.PERIOD,JTK.LAG,JTK.AMP)
  })
  res <- as.data.frame(t(res))
  bhq <- p.adjust(unlist(res[,1]),"BH")
  res <- cbind(bhq,res)
  colnames(res) <- c("BH.Q","ADJ.P","PER","LAG","AMP")
  results <- cbind(annot,res,data)
  results <- results[order(res$ADJ.P,-res$AMP),]
})
print(st)

save(results,file=paste("JTK",project,"rda",sep="."))
write.table(results,file=paste("JTK",project,"txt",sep="."),row.names=F,col.names=T,quote=F,sep="\t")

## MetaCycle

In [ ]:
library(MetaCycle, lib.loc="C:/Users/Arthur/Documents/R/win-library/3.5")

In [ ]:
meta2d(infile="wtcounts.csv", filestyle="csv", outdir="WT_metacycle", timepoints=rep(seq(0, 18, by=6), each=5), 
       #this specify the range of time points, in this case starting at ct0 ending at ct18, 
       #sampled 6 hours apart, with 5 replciates per timepoint
       minper=12, maxper=24,
       cycMethod=c("JTK","LS"), outIntegration="both") 
#outIntergration = "both" will make metacycle almagamate results from JTK & LS

## RAIN

In [ ]:
library(rain)
wtcounts <- read.delim("C:/Users/Arthur/Desktop/sox2revision/data/wtcounts.txt", row.names=1)
View(wtcounts)
results<-rain(t(wtcounts),deltat=6,period=24,nr.series=5,peak.border=c(0.2,0.8), verbose = TRUE)


In [ ]:
kocounts<-read.delim("C:/Users/Arthur/Desktop/sox2revision/data/kocounts.txt", row.names=1)
head(kocounts)
results<-rain(t(kocounts),deltat=6,period=24,nr.series=5,peak.border=c(0.2,0.8),verbose=TRUE)

## ARSER using MetaCycle

ARSER cannot handle biological replicates, so let's take the mean/median of the replicates. Another way is to concatenate all the replicates into 1 single time series. This is outrageous in principle but I tried it nonetheless.

In [ ]:
meta2d(infile="wtmedian.txt", filestyle="txt", outdir="a3_wtmedian", timepoints=seq(0, 18, by=6), 
       #this specify the range of time points, in this case starting at ct0 ending at ct18, 
       cycMethod=c("JTK","LS","ARS"), outIntegration="both") 
#outIntergration = "both" will make metacycle almagamate results from JTK & LS & ARS

In [ ]:
meta2d(infile="komedian.txt", filestyle="txt", outdir="a3_komedian", timepoints=seq(0, 18, by=6), 
       #this specify the range of time points, in this case starting at ct0 ending at ct18, 
       cycMethod=c("JTK","LS","ARS"), outIntegration="both") 
#outIntergration = "both" will make metacycle almagamate results from JTK & LS & ARS

## DESeq2 starting with summarizedExperiment

In [ ]:
library(DESeq2)
load("C:/Users/Arthur/Desktop/sox2revision/Rob/imadethis.RData") 
#I already put in most of the colData into this RData but this is still unfiltered raw count

load("C:/Users/Arthur/Desktop/sox2revision/Rob/cpmfilterseitem.RData")
#this is the filtered version with cpm >0.1
#I first took the raw count excel file and calculate CPM for each gene in each library, took out those below the threshold and 
#saved as seperate file, say "summarizedExperimentwfilter.csv"

load("C:/Users/Arthur/Desktop/sox2revision/Rob/imadethis.RData") #original SE
sef<-read.csv("summarizedExperimentwfilter.csv") #filtered SE
rownames(sef)<-sef[,1]
b<-sef$X
sef$X<-NULL
keep<-rownames(assay(se)) %in% b
nse<-se[keep,]

In [ ]:
nse$group<-factor(c("f","h","h","b","c","a","g","d",
                                    "h","e","d","d","c","e","g","d",
                                    "a","e","c","e","g","a","f","b",
                                    "d","b","f","a","b","c","h","b",
                                    "f","g","g","a","e","c","f","h"))
colData(nse)

#arrange the column by group and by replicate number
onse<-nse[, order(nse$rep)]
oonse<-onse[, order(onse$group)]
colData(oonse)
save(oonse, file="summarizedExperiment_v1.RData") #current raw count summarizedExperiment


#design1: subset by timepoint
dds<- DESeqDataSet( se[, se$time == "0"], design = ~genotype)
#dds <- dds[ rowSums(counts(dds)) > 1, ] 
dds<- DESeq(dds)
res0<- results(dds)
write.csv(res0, "fDESeq2_subset/ct0.csv")

dds<- DESeqDataSet( se[, se$time == "6"], design = ~genotype)
#dds <- dds[ rowSums(counts(dds)) > 1, ]
dds<- DESeq(dds)
res6<- results(dds)
write.csv(res6, "fDESeq2_subset/ct6.csv")

dds<- DESeqDataSet( se[, se$time == "12"], design = ~genotype)
#dds <- dds[ rowSums(counts(dds)) > 1, ]
dds<- DESeq(dds)
res12<- results(dds)
write.csv(res12, "fDESeq2_subset/ct12.csv")

dds<- DESeqDataSet( se[, se$time == "18"], design = ~genotype)
#dds <- dds[ rowSums(counts(dds)) > 1, ]
dds<- DESeq(dds)
res18<- results(dds)
write.csv(res18, "fDESeq2_subset/ct18.csv")


#design2: group genotype+timepoint
dds<-DESeqDataSet(oonse, design = ~group)
#dds <- dds[ rowSums(counts(dds)) > 1, ] #this filter out really low count genes
dds<-DESeq(dds)
res0 <- results(dds, contrast=c("group","e","a"))
res6 <- results(dds, contrast=c("group","f","b"))
res12 <- results(dds, contrast=c("group","g","c"))
res18 <- results(dds, contrast=c("group","h","d"))
write.csv(res0, "fDESeq2_group_AvB/ct0.csv")
write.csv(res6, "fDESeq2_group_AvB/ct6.csv")
write.csv(res12, "fDESeq2_group_AvB/ct12.csv")
write.csv(res18, "fDESeq2_group_AvB/ct18.csv")

#design3
# recommended by Michael Love on https://support.bioconductor.org/p/112118/
dds<-DESeqDataSet(nse, design = ~time+time:genotype)
dds<-DESeq(dds)
resultsNames(dds)
res0=results(dds, name="time0.genotypeKO")
res6=results(dds, name="time6.genotypeKO")
res12=results(dds, name="time12.genotypeKO")
res18<-results(dds, name="time18.genotypeKO")
write.csv(res0,file="hopetogod.csv")
write.csv(res6,file="hopetogod1.csv")
write.csv(res12,file="hopetogod12.csv")
write.csv(res18,file="hopetogod18.csv")

## Heatmap of DEGs expression across timepoint

In [ ]:
vsd<-vst(dds,blind=FALSE) #transform the dds using VST

#order the columns into WT0, WT6, WT12, WT18, KO0, KO6, KO12, KO18
order(nse$group)
ovsd<-assay(vsd)[,order(nse$group)]
head(ovsd)

#filter out the nonDEGs
DEG<-read.csv("deseq2_237degs_fdr.05.csv", header=TRUE) #list of ensemblID for the 237 DEGs
filter<-rownames(vsd)%in%DEG$ensemblID
degonly<-vsd[filter,]

#make a matrix of expression minus mean expression
mat<-assay(degonly)-rowMeans(assay(degonly))
anno <- as.data.frame(colData(vsd)[, c("genotype","time_h")])
pheatmap(mat, annotation_col = anno, cluster_cols = FALSE)

#this is borrowed from https://slowkow.com/notes/heatmap-tutorial/ to make each color represent the same number of blocks
quantile_breaks <- function(xs, n = 10) {
breaks <- quantile(xs, probs = seq(0, 1, length.out = n))
breaks[!duplicated(breaks)]
}
mat_breaks <- quantile_breaks(mat, n = 11)
res<-pheatmap(mat, annotation_col = anno, cluster_cols = FALSE, color= viridis(length(mat_breaks) - 1),
breaks= mat_breaks, show_colnames = F, show_rownames = F)

#get the rownames of the tree in order of the clustering
geneid <- mat[c(res$tree_row[["order"]]),]

#look at the dendrogram
plot(res$tree_row) 
abline(h=3, col="red", lty=2, lwd=2) #from this graph we can roughly decide how many clusters we want to group the genes into

#here I picked the h=3 to cut the trees into 9 branches
write.csv(data.frame(sort(cutree(res$tree_row, h=3))), file="cutting_tree.csv")

#this is how the heatmap looks like if it is divided by the main branches
res<-pheatmap(mat, annotation_col = anno, cluster_cols = FALSE, color= viridis(length(mat_breaks)-1),
    breaks= mat_breaks, show_colnames = F, show_rownames = F, cutree_rows = 9)

#another way to cut tree
mt.clust <- cbind(mat, cluster = cutree(res$tree_row, k = 9))

#sorting the dendrogram
library(dendsort)
library(dendextend)
sort_hclust <- function(...) as.hclust(dendsort(as.dendrogram(...)))
mat_cluster_cols <- sort_hclust(res$tree_row)
res<-pheatmap(mat, annotation_col = anno, cluster_cols = FALSE, cluster_rows = rev(mat_cluster_cols), 
              color= viridis(length(mat_breaks)-1), breaks= mat_breaks, show_colnames = F, show_rownames = F, cutree_rows = 9)

In [ ]:
#plot heatmap for rhythmic genes using VST transformed value

#plotting both_rhythmic gene set and sort by wt phase
um<-read.csv("finalizing_v1/meta2d/both_rhythmic.csv")
rowstokeep<-um[,"CycID"]
both_rg<-vsd[rowstokeep,]
dim(assay(both_rg))
mat<-assay(both_rg)-rowMeans(assay(both_rg))
anno <- as.data.frame(colData(vsd)[, c("genotype","time_h")])
omat<-mat[order(um$meta2d_phase),] #sort by WT phase
mat_breaks <- quantile_breaks(omat, n = 21)
res<-pheatmap(omat, annotation_col = anno,cluster_rows = F, cluster_cols = FALSE, color= viridis(length(mat_breaks) - 1),
breaks= mat_breaks, show_colnames = F, show_rownames = F)

justwt<-read.csv("finalizing_v1/meta2d/wt_rhythmic_genes.csv") #this contain genes that are rhythmic in both genotype and only in wt
justwt<-justwt[justwt$rhythmic=="wt",] #filter out genes that are also rhythmic in KO as they have been plotted above
rowstokeep<-justwt[,"CycID"]
wt_rg<-vsd[rowstokeep,]
dim(assay(wt_rg))
mat<-assay(wt_rg)-rowMeans(assay(wt_rg))
anno <- as.data.frame(colData(vsd)[, c("genotype","time_h")])
omat<-mat[order(justwt$meta2d_phase),]
mat_breaks <- quantile_breaks(omat, n = 21)
res<-pheatmap(omat, annotation_col = anno,cluster_rows = F, cluster_cols = FALSE, color= viridis(length(mat_breaks) - 1),
breaks= mat_breaks, show_colnames = F, show_rownames = F)


justko<-read.csv("finalizing_v1/meta2d/ko_rhythmic_genes.csv") #contain genes that are rhythmic in both genotype and only in ko
justko<-justko[justko$rhythmic=="ko",] #filter out genes that are also rhythmic in WT as they have been plotted above
rowstokeep<-justko[,"CycID"]
ko_rg<-vsd[rowstokeep,]
dim(assay(ko_rg))
mat<-assay(ko_rg)-rowMeans(assay(ko_rg))
anno <- as.data.frame(colData(vsd)[, c("genotype","time_h")])
omat<-mat[order(justko$meta2d_phase),]
mat_breaks <- quantile_breaks(omat, n = 21)
res<-pheatmap(omat, annotation_col = anno,cluster_rows = F, cluster_cols = FALSE, color= viridis(length(mat_breaks) - 1),
breaks= mat_breaks, show_colnames = F, show_rownames = F)

## Correcting for transcript length

Using normalized count for constructing heatmap and GO enrichment analysis is not optimal as longer genes will have more reads than shorter genes. To correct for this I used *biomaRT* package to fetch transcript length for each gene. As some genes have splice variants, they will have multiple transcript length entries.

In [ ]:
library(biomaRT)
ensembl=useMart("ENSEMBL_MART_ENSEMBL")
ensembl1<-useDataset("mmusculus_gene_ensembl",mart=ensembl)
transcriptlen=getBM(attributes = c('ensembl_gene_id',"transcript_length"),
                filters="ensembl_gene_id",
                values=ensemblID, 
#ensemblID is a list of genes that passed the expression cutoff I set, as of 20180812 that is CPM>0.1 in at least 5 libraries(out of 40)
                mart=ensembl1)
write.csv(transcriptlen, file="transcript_length.csv")

## Graphs for MetaCycle Analysis

In [ ]:
#rose plot for rhythmic genes phase
#dodge
p<-ggplot(circledata, aes(x = meta2d_phase, color = genotype, fill=genotype)) + geom_histogram(breaks = seq(0, 24), 
position="dodge")+ coord_polar(start = 0) + theme(legend.position = "top") +ylab("Count") + ggtitle("phase") + 
scale_x_continuous("", limits = c(0, 24), breaks = seq(0, 24), labels = seq(0, 24))

#stack
p<-ggplot(circledata, aes(x = meta2d_phase, color = genotype, fill=genotype)) + geom_histogram(breaks = seq(0, 24), 
position=position_stack(reverse=TRUE), color="black")+ coord_polar(start = 0) + theme(legend.position = "top") +ylab("Count") + ggtitle("phase") + 
scale_x_continuous("", limits = c(0, 24), breaks = seq(0, 24), labels = seq(0, 24))

p+geom_vline(mapping = NULL, data = NULL, xintercept=c(0,6,12,18) , na.rm = FALSE,
show.legend = NA, color="red")+scale_y_continuous(limits = c(0,150))

In [ ]:
#distribution of relative amplitude of rhythmic genes in either genotype
p<-ggplot(circledata, aes(x = meta2d_rAMP, color = rhythmic, fill=rhythmic)) + 
geom_histogram(position=position_stack(reverse=TRUE), color="black", breaks = seq(0,1.5,0.05)) + 
theme(legend.position = "top") +ylab("count") + ggtitle("relative amplitude") + 
scale_y_continuous(limits=c(0,120), breaks=seq(0,180,10)) + 
scale_x_continuous(breaks=seq(-.25,1.5,.25))

pg <- ggplot_build(p)

q<-ggplot(wtcircledata, aes(x = meta2d_rAMP, color = rhythmic, fill=rhythmic)) +
geom_histogram(position=position_stack(reverse=TRUE), color="black", breaks = seq(0,1.5,0.05)) +
theme(legend.position = "top") +ylab("count") + ggtitle("relative amplitude") + 
scale_y_continuous(limits=c(0,120), breaks=seq(0,180,10)) + 
scale_x_continuous(breaks=seq(-.25,1.5,.25))

qg<-ggplot_build(q)

In [ ]:
#distribution of period of rhythmic genes in either genotype
p<-ggplot(circledata, aes(x = meta2d_period, color = rhythmic, fill=rhythmic)) +
geom_histogram(position=position_stack(reverse=TRUE), color="black", breaks = seq(12,24,0.5)) +
theme(legend.position = "top") +ylab("count") + ggtitle("period") +
scale_y_continuous(limits=c(0,150),breaks=seq(0,180,10)) +
scale_x_continuous(breaks=seq(11,24,.5))

q<-ggplot(wtcircledata, aes(x = meta2d_period, color = rhythmic, fill=rhythmic)) +
geom_histogram(position=position_stack(reverse=TRUE), color="black", breaks = seq(12,24,0.5)) +
theme(legend.position = "top") +ylab("count") + ggtitle("period") +
scale_y_continuous(limits=c(0,150), breaks=seq(0,180,10)) +
scale_x_continuous(breaks=seq(11,24,.5))

In [ ]:
#phase against relative amplitude for all rhythmic genes in either genotype
p<-ggplot(circledata, aes(x = meta2d_phase, y= meta2d_rAMP, color = rhythmic, shape=rhythmic)) + 
geom_point()+ theme(legend.position = "top") +ylab("rAMP") + ggtitle("")+
coord_polar(start=0) + 
scale_y_continuous(limits = c(0,1.5), breaks=seq(0,4,.25))+ 
scale_x_continuous("", limits = c(0, 24), breaks = seq(0, 24), labels = seq(0, 24))

q<-ggplot(wtcircledata, aes(x = meta2d_phase, y= meta2d_rAMP, color = rhythmic, shape=rhythmic)) + 
geom_point()+ theme(legend.position = "top") + ylab("rAMP") + ggtitle("") + 
coord_polar(start=0)+
scale_y_continuous(limits = c(0,1.5), breaks= seq(0,4,.25))+
scale_x_continuous("", limits = c(0, 24), breaks = seq(0, 24), labels = seq(0, 24))

In [ ]:
#phase against period for all rhythmic genes in either genotype
p<-ggplot(circledata, aes(x = meta2d_phase, y= meta2d_period, color = rhythmic, shape=rhythmic)) + 
geom_point()+ theme(legend.position = "top") +ylab("Period") + ggtitle("") + 
coord_polar(start=0) + 
scale_x_continuous("", limits = c(0, 24), breaks = seq(0, 24), labels = seq(0, 24))+
expand_limits(y=0)

q<-ggplot(wtcircledata, aes(x = meta2d_phase, y= meta2d_period, color = rhythmic, shape=rhythmic)) + 
geom_point()+ theme(legend.position = "top") +ylab("Period") + ggtitle("") + 
coord_polar(start=0) + 
scale_x_continuous("", limits = c(0, 24), breaks = seq(0, 24), labels = seq(0, 24))+
expand_limits(y=0)

In [ ]:
#focus on the both_rhythmic gene set: contrast the attributes for each genotype
#these are KO-WT
data<- read.csv("intermediate/both_rhythmic_difference.csv")
l<-ggplot(data, aes(x=Gene, y=diff_meta2d_period))+geom_point()+theme(axis.text.x = element_text(angle=90, size=8))
j<-ggplot(data, aes(x=Gene, y=diff_meta2d_rAMP))+geom_point()+theme(axis.text.x=element_text(angle=90, size=8))

#for the diff_phase, it is neccessary to adjust the value for the genes with >12 or <12 diff_phase
data1<-data
data1[data1$diff_meta2d_phase>12,"diff_meta2d_phase"]<-data1[data1$diff_meta2d_phase>12,"diff_meta2d_phase"]-24
data2<-data1
data2[-12>data2$diff_meta2d_phase,"diff_meta2d_phase"]<-data2[-12>data2$diff_meta2d_phase,"diff_meta2d_phase"]+24
k<-ggplot(data2, aes(x=Gene, y=diff_meta2d_phase))+geom_point()+theme(axis.text.x=element_text(angle=90, size=8))

#use when neccessary + geom_hline(aes(yintercept=0),color="red", linetype="solid", size=1)

In [ ]:
#plot genes that are rhythmic in both genotypes with phase as x axis
newcsv<-read.csv("intermediate/both_rhythmic.csv")
n<-ggplot(newcsv, aes(x = meta2d_phase, y= meta2d_rAMP, color = CycID, shape=genotype)) + 
geom_point(alpha=0.6)+ theme(legend.position = "top") +ylab("rAMP") + ggtitle("")+coord_polar(start=0)+
scale_y_continuous(limits = c(0,1.5), breaks= seq(0,4,.25))+
scale_x_continuous("", limits = c(0, 24), breaks = seq(0, 24), labels = seq(0, 24))+guides(color=FALSE)

m<-ggplot(newcsv, aes(x = meta2d_phase, y= meta2d_period, color = CycID, shape=genotype)) +
geom_point(alpha=0.6)+
theme(legend.position = "top") +
ylab("Period") + ggtitle("") + coord_polar(start=0) +
scale_y_continuous(breaks= seq(0,36,6))+
scale_x_continuous("", limits = c(0, 24), breaks = seq(0, 24), labels = seq(0, 24))+
guides(color=FALSE) +expand_limits(y=0)

In [ ]:
#plot phase/period/genes that are rhythmic in both genotypes
n<-ggplot(newcsv, aes(x = Gene, y= meta2d_rAMP, color = genotype)) +
geom_point(alpha=0.6)+ theme(legend.position = "top", axis.text.x=element_text(angle=90, size=9)) +ylab("rAMP") +
scale_y_continuous(limits = c(0,1.5), breaks= seq(0,4,.25))
n+scale_color_brewer(palette="Set1")

m<-ggplot(newcsv, aes(x = Gene, y= meta2d_period, color = genotype)) +
    geom_point(alpha=0.6)+ theme(legend.position = "top", axis.text.x=element_text(angle=90, size=9)) +ylab("Period") +
    scale_y_continuous(limits = c(12,24), breaks= seq(0,24,1))
m+scale_color_brewer(palette="Set1")

o<-ggplot(newcsv, aes(x = Gene, y= meta2d_phase, color = genotype)) +
    geom_point(alpha=0.6)+ theme(legend.position = "top", axis.text.x=element_text(angle=90, size=9)) +ylab("Phase") +
    scale_y_continuous(limits = c(0,24), breaks= seq(0,24,1))
o+scale_color_brewer(palette="Set1")


In [ ]:
#function to display multiple ggplot side by side
#http://www.cookbook-r.com/Graphs/Multiple_graphs_on_one_page_(ggplot2)/
multiplot <- function(..., plotlist=NULL, file, cols=1, layout=NULL) {
    library(grid)
    
    # Make a list from the ... arguments and plotlist
    plots <- c(list(...), plotlist)
    
    numPlots = length(plots)
    
    # If layout is NULL, then use 'cols' to determine layout
    if (is.null(layout)) {
        # Make the panel
        # ncol: Number of columns of plots
        # nrow: Number of rows needed, calculated from # of cols
        layout <- matrix(seq(1, cols * ceiling(numPlots/cols)),
                         ncol = cols, nrow = ceiling(numPlots/cols))
    }
    
    if (numPlots==1) {
        print(plots[[1]])
        
    } else {
        # Set up the page
        grid.newpage()
        pushViewport(viewport(layout = grid.layout(nrow(layout), ncol(layout))))
        
        # Make each plot, in the correct location
        for (i in 1:numPlots) {
            # Get the i,j matrix positions of the regions that contain this subplot
            matchidx <- as.data.frame(which(layout == i, arr.ind = TRUE))
            
            print(plots[[i]], vp = viewport(layout.pos.row = matchidx$row,
                                            layout.pos.col = matchidx$col))
        }
    }
}

## Misc

In [ ]:
#arrange the normalized count: each column represent 1 gene, assign the genotype and time point for each row as a new column
pltList<-list()
for (s in 1:20)
    local({
        s<-s
        time<-tsubset$time
        genotype<-tsubset$genotype
        x<-tsubset[,s]
        p<-data.frame(time,x,genotype)
        print(s)
        pltList[[s]]<<-print(ggplot(p, aes(x=p$time,y=p$x,shape=genotype, color=genotype))+
            geom_point()+scale_shape_manual(values=c(1,17))+
            theme(legend.position = "none", axis.title.x=element_blank(),axis.title.y=element_blank()))
})
multiplot(plotlist=pltList, cols=5)